This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

None


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import numpy as np

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
response = requests.get(url)
#check import was with no errors
response.status_code

200

In [4]:
#save data as a dictionary 
json_data = response.json()

## Inspecting the imported data

In [5]:
#double check that the data is a dictionary 
print(type(json_data))
#get the names of keys
print(json_data.keys())

<class 'dict'>
dict_keys(['dataset_data'])


In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

In [79]:
#we expect to see nested dictionaries 
#so this is to get the names of the inner keys
print(json_data['dataset_data'].keys())
#time period is from 2000 to 2020
print(json_data['dataset_data']['start_date'])
print(json_data['dataset_data']['end_date'])
print(json_data['dataset_data']['data'][0:2])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
2000-06-07
2020-12-01
[['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None], ['2020-11-30', 111.0, 113.6, 111.0, 112.1, None, 315.0, 35111.5, None, None, None]]


## 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [9]:
#data for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
url_2017 = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
response_2017 = requests.get(url_2017)
#double check that import is without errors
response_2017.status_code

200

## 2. Convert the returned JSON object into a Python dictionary.

In [10]:
#convert JSON object to python dictionary 
data_2017 = response_2017.json()
#confirm JSON object is a python dictionary 
print(type(data_2017))
#check keys
print(data_2017['dataset_data'].keys())
#check we have the right data set
print(data_2017['dataset_data']['start_date'])
print(data_2017['dataset_data']['end_date'])

<class 'dict'>
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
2017-01-01
2017-12-31


## 3.  Calculate what the highest and lowest opening prices were for the stock in this period.

In [11]:
#Problem: we are looking for the opening prices but we don't actually oepning prices as a key
#Solution: we need to explore the keys more as we know this will be a nested dictionary

In [12]:
print(data_2017['dataset_data']['column_names'])
print(data_2017['dataset_data']['data'][0:2])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None]]


This shows us that the second column is the opening prices, so index 1.

In [13]:
#list of all the opening prices for 2017
opening_prices = []
for values in data_2017['dataset_data']['data']:
    opening_prices.append(values[1])
#check if there are any values that are not numbers
#None in opening_prices

In [14]:
#list of only opening prices which are floats
opening_prices_floats = []
for items in opening_prices:
    if type(items)== float:
        opening_prices_floats.append(items)
print("The minimum opening price for Carl Zeiss Meditec in 2017 was $" + str(min(opening_prices_floats)))
print("The maximum opening price for Carl Zeiss Meditec in 2017 was $" + str(max(opening_prices_floats)))

The minimum opening price for Carl Zeiss Meditec in 2017 was $34.0
The maximum opening price for Carl Zeiss Meditec in 2017 was $53.11


## 4. What was the largest change in any one day (based on High and Low price)?

In [15]:
daily_high_prices = []
daily_low_prices = []
for values in data_2017['dataset_data']['data']:
    daily_high_prices.append(values[2])
    daily_low_prices.append(values[3])

In [16]:
'''
Run this cell only to run checks. 

#check if there are any none values in the daily high prices and daily low prices
print(None in daily_high_prices)
print(None in daily_low_prices)
#make sure the lenghts of the two lists are the same so we can subtract corresponding values
print(len(daily_high_prices))
print(len(daily_low_prices))

'''

'\nRun this cell only to run checks. \n\n#check if there are any none values in the daily high prices and daily low prices\nprint(None in daily_high_prices)\nprint(None in daily_low_prices)\n#make sure the lenghts of the two lists are the same so we can subtract corresponding values\nprint(len(daily_high_prices))\nprint(len(daily_low_prices))\n\n'

In [17]:
daily_change = []
for values in data_2017['dataset_data']['data']:
    if type(values[2]) == float and type(values[3]) == float:
        change_in_price =(values[2] - values[3])
        daily_change.append(change_in_price)
max_change = "%.2f"%max(daily_change)                
           
print("The largest change in stock prices for any one day for Carl Zeiss Meditec in 2017 was $" + str(max_change))

The largest change in stock prices for any one day for Carl Zeiss Meditec in 2017 was $2.81


## 5. What was the largest change between any two days (based on Closing Price)?

In [18]:
closing_prices_floats = []
date = []
for values in data_2017['dataset_data']['data']:
    #we only want the dates where the closing and opening prices are floats
    if type(values[1])== float and type(values[4])==float:
        date.append(values[0])
        closing_prices_floats.append(values[4])
#len(closing_prices_floats)
#len(date)
#len(opening_prices_floats)

So, now we have lists of opening, closing prices, and dates. To calculate the price change over two days we need to subtract the closing price of a day from the opening prices of the previous date. 
At the moment the dates are in decending order: starting from December and going down to January, as are the values for opening and closing. 

Assumption: we are looking for the maximum change between two consecutive dates. 

In [19]:
date.reverse()
opening_prices_floats.reverse()
closing_prices_floats.reverse()

In [20]:
change_over_cons_days = []
for i in range(len(closing_prices_floats)):
    if i <= 250:
        change = closing_prices_floats[i+1] - opening_prices_floats[i]
        change_over_cons_days.append(change)
max_change_cons_days = "%.2f"%max(change_over_cons_days)        
print("The largest change in stock prices over two consecutive days for Carl Zeiss Meditec in 2017 was $" + str(max_change_cons_days))

The largest change in stock prices over two consecutive days for Carl Zeiss Meditec in 2017 was $2.17


In [75]:
change_over_cons_days = []
for i in range(len(closing_prices_floats)):
    if i != 0:
        change = opening_prices_floats[i] - closing_prices_floats[i-1]
        change_over_cons_days.append(change)
max_change_cons_days = "%.2f"%max(change_over_cons_days)        
print("The largest change in stock prices over any two days for Carl Zeiss Meditec in 2017 was $" + str(max_change_cons_days))

The largest change in stock prices over any two days for Carl Zeiss Meditec in 2017 was $0.78


## 6. What was the average daily trading volume during this year?

In [80]:
#checking where to find the traded volume
#print(data_2017['dataset_data']['column_names'])
#print(data_2017['dataset_data']['data'][0:2])
#print(data_2017['dataset_data']['column_names'][6])

In [82]:
#Traded volume is index 6. 
daily_traded_volumes = []
#extract traded values
for values in data_2017['dataset_data']['data']:
    daily_traded_volumes.append(values[6])

In [ ]:
'''
Run this cell to check the type of data in daily_traded_volumes is only float 
types = []
for item in daily_traded_volumes:
    types.append(type(item))
print(set(types))
'''

In [51]:
#Average is the sum of all the elements in the list / the total number of elements in the list 
average_daily_traded_volume = sum(daily_traded_volumes)/len(daily_traded_volumes)
#to neaten up to 2dp
rounded_average = "%.2f"%average_daily_traded_volume        
print("The average daily volume of stocks for Carl Zeiss Meditec in 2017 was " + str(rounded_average))

The average daily volume of stocks for Carl Zeiss Meditec in 2017 was 89124.34


## 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [72]:
#to work out the median we need to first sort the list in ascending order
ascending_traded_volumes = sorted(daily_traded_volumes) 
#we want to find the item in the list which is the middle index: so len(list)/2 should give us the middle value.
#we know the length of the list will be 255 so we need to round up for the 128th index
median = ascending_traded_volumes[round(len(ascending_traded_volumes)/2)]
print("The median daily traded volume of stocks for Carl Zeizz Meditec in 2017 was " + str(median))

The median daily traded volume of stocks for Carl Zeizz Meditec in 2017 was 76600.0
